# GBDT Stock Agent (Colab GPU)

このノートブックは Local->Colab 引き継ぎ済み run を段階実行する固定フローです。
- 実行方式: `/content` で処理 + Driveは同期専用
- 停止/再開: 各ステージセル単位
- 必須: Colab移行前レポート承認後のみ実行

In [ ]:
# 0) Runtime setup
from google.colab import drive
drive.mount('/content/drive')

!cd /content && rm -rf gbdt-stock-agent && git clone https://github.com/shonpyama/gbdt-stock-agent.git
%cd /content/gbdt-stock-agent
!python -m pip install --upgrade pip
!pip install -e .

In [ ]:
# 1) Restore migration bundle (Local -> Colab)
# 例: /content/drive/MyDrive/gbdt-stock-agent/migration/run_20260221_092119Z_021cc3de_c507b2ff.zip
ARCHIVE = '/content/drive/MyDrive/gbdt-stock-agent/migration/run_20260221_092119Z_021cc3de_c507b2ff.zip'
!python -m gbdt_agent.cli migrate restore --archive "$ARCHIVE"

In [ ]:
# 2) API key + fast sync (10 min)
import os
from getpass import getpass
from pathlib import Path
from gbdt_agent.colab import setup_fast_colab_persistence

if not os.environ.get('FMP_API_KEY'):
    os.environ['FMP_API_KEY'] = getpass('FMP_API_KEY: ')

DRIVE_PATH = Path('/content/drive/MyDrive/gbdt-stock-agent')
sync_handle = setup_fast_colab_persistence(drive_path=DRIVE_PATH, interval_seconds=600)
print('sync started:', DRIVE_PATH)

In [ ]:
# 3) Preflight
!python -m gbdt_agent.cli preflight --conf conf/default.yaml

In [ ]:
# 4) Stage runner (resume-safe)
from pathlib import Path
from gbdt_agent.orchestrator import run_pipeline

PROJECT_DIR = Path('/content/gbdt-stock-agent')
CONF = PROJECT_DIR / 'conf/default.yaml'

def run_to(stage: str, force_unlock: bool = False):
    run_id = run_pipeline(
        project_dir=PROJECT_DIR,
        conf_path=CONF,
        resume=True,
        force_unlock=force_unlock,
        stop_after_stage=stage,
    )
    print('run_id=', run_id, 'stage=', stage)
    return run_id

In [ ]:
# 5) Stage 10-20
run_id = run_to('stage_10_data_ready')
run_id = run_to('stage_20_validation_passed')

In [ ]:
# 6) Stage 30-40
run_id = run_to('stage_30_features_ready')
run_id = run_to('stage_40_split_leakcheck_passed')

In [ ]:
# 7) Stage 50-60
run_id = run_to('stage_50_models_trained')
run_id = run_to('stage_60_predictions_ready')

In [ ]:
# 8) Stage 70-80
run_id = run_to('stage_70_backtest_ready')
run_id = run_to('stage_80_report_ready')

In [ ]:
# 9) Manual sync checkpoint (phase end)
!python -m gbdt_agent.cli colab sync --drive-path /content/drive/MyDrive/gbdt-stock-agent

In [ ]:
# 10) Required report for hand-off/review
!python -m gbdt_agent.cli report --run-id $run_id
!python -m gbdt_agent.cli transition-report --run-id $run_id --target colab